## Тип документа

Тип документа указывается в шапке документа (в верхней части документа).

В рамках задания документ может относиться к одному из следующих типов:

* «федеральный закон»
* «постановление»
* «приказ»
* «распоряжение»
* «закон»
* «указ»

Обратите внимание, что в перечне присутствуют два похожих, но отдельных типа: «закон» и «федеральный закон».

Поле type должно содержать строку, обозначающую какой-то один из указанных типов, это задача классификации.

**Оценка качества:** Требуется предсказать один из шести классов. Метрика качества – макро f1-мера.

Признаки:

* количество "федеральный", "закон", "постановление", "приказ", "распоряжение", "указ", "федеральный закон" в тексте;
* позиция, в которой встретилось каждое из этих слов;
* бинарный признак - если какое-то из слов встретилось и написано капсом;
* бинарный признак - если слово единственное в данной строке (учитываем слова из букв);
* бинарный признак - если слово первое в строке (документе);
* учет опечаток в словах??? - м. б. находить расстояние между словами (если слово написано капсом и одно в строке, то можно его сравнить со словами из нашего множества)
* можно добавить еще поиск слов "постановляет" и т. д.

In [1]:
import re
import json
import numpy as np
from eval_module import quality

In [2]:
train = []
labeled = []
names = []
with open("train/gold_labels.txt", "r") as read_file:
    for doc_info in read_file.readlines():
        doc_dict = json.loads(doc_info)
        docname = 'train/txts/' + doc_dict['id'] + '.txt'
        names.append(docname)
        with open(docname, 'r') as f:
            train.append(f.read())
        labeled.append(doc_dict['label'])
names = np.array(names)
true_types = np.array([x['type'] for x in labeled])

In [195]:
def preprocess(doc):
    doc = '\n' + doc.lower()
    doc = re.sub(r'[нп]рика[з3]', 'приказ', doc)
    doc = re.sub('ука3', 'указ', doc)
    return doc

In [433]:
exprs = ['закон\s|федеральный закон', 'приказ|истрировано', 'указ', 'распоряжение', 'постановление']

def find_best_match(types, doc):
    # matches: закон приказ указ распоряжение постановление
    ids = np.where(types)[0]
    expr = ""
    for i in ids:
        expr = f"{expr}|{exprs[i]}"
    if expr:
        expr = expr[1:]
    else:
        return expr
    expr = re.compile(expr)
    
    matches = expr.findall(doc)
    if matches[0].strip() in ['закон', 'федеральный закон', 'указ', 'распоряжение', 'постановление']:
        return matches[0].strip()
    else:
        return 'приказ'

In [434]:
type_reg_exprs = [
    re.compile(r'закон|федеральный закон'),
    re.compile(r'приказ|истрировано'),
    re.compile(r'указ'),
    re.compile(r'распоряжение'),
    re.compile(r'постановление')
]

def extract_type(doc):
    doc = preprocess(doc)
    if re.search('костромская областная дума', doc):
        return 'закон'
    matches = np.array([expr.search(doc) for expr in type_reg_exprs])
    return find_best_match(matches, doc)

In [435]:
def predict(test):
    results = []
    for doc in test:
        doc_type = extract_type(doc)
        prediction = {"type": doc_type,
                      "date": "",
                      "number": "",
                      "authority": "",
                      "name": ""}
        results.append(prediction)
    return results

In [436]:
predicted = predict(train)
quality(predicted, labeled)

{'date_accuracy': 0.0,
 'number_accuracy': 0.0,
 'type_f1_score': 1.0,
 'name_jaccard': 0.0,
 'authority_jaccard': 0.0,
 'subtasks_improves': 1}

In [437]:
predicted_labels = np.array(predicted)
test_labels = np.array(labeled)
for doc_type in ['федеральный закон', 'постановление', 'приказ', 'распоряжение','закон', 'указ']:
    ids = np.where(true_types == doc_type)
    print(f"{doc_type} ({ids[0].shape[0]}): {quality(predicted_labels[ids], test_labels[ids])['type_f1_score']}")

федеральный закон (192): 1.0
постановление (2459): 1.0
приказ (77): 1.0
распоряжение (430): 1.0
закон (199): 1.0
указ (632): 1.0


In [438]:
doc_type = 'указ'
ids = np.where(true_types == doc_type)

y_test = [x['type'] for x in test_labels[ids]]
y_pred = [x['type'] for x in predicted_labels[ids]]

wrong_docs = []
i = 0
for test, pred in zip(y_test, y_pred):
    if test != pred:
        wrong_docs.append((names[ids][i], test, pred))
    i += 1
    
for doc_name, test, pred in wrong_docs:
    with open(doc_name, 'r') as f:
        print(f"right value = {test}")
        print(f"wrong value = {pred}")
        print(f"docname = {doc_name}")
        print(f.read())
        print("=====")